# MONET2030 - Analysis

In [ ]:
# Stdlib imports
import re
from pathlib import Path
from collections import namedtuple

# 3rd party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colormaps as cm

# Local imports
from pymonet import monet_scraper as scraper
from pymonet import monet_processor as processor
from pymonet import monet_consts as const
from sipi_da_utils import utils, tsa, plot

## 1) Get Web Data

In [ ]:
scraper_pipeline = scraper.MonetLoader()
raw_data = await scraper_pipeline.load()

## 2) Process/Transform Data

In [ ]:
pipeline = processor.TransformationPipeline(raw_data,
                                            scraper_pipeline.indicators_metatable,
                                            scraper_pipeline.observables_metatable
                                           )
final_output = pipeline.run()

## 3) Visual inspection

In [ ]:
figsaxes = pipeline.create_inspection_plots(create='all', write=True)

## 4) Analysis

### 4.1) Setup & Preparations

In [ ]:
metric2capital_map = pipeline.stages[1].additional_results['metric_id2name_map'][["metric_id", "metric_name", "capital - primary"]].set_index("metric_id")
metric2capital_map.head()

In [ ]:
four_caps = set([c for c in metric2capital_map["capital - primary"]])
four_caps

In [ ]:
monet_data = pipeline.stages[2].output.copy()
monet_clean = pipeline.stages[3].output.copy()

### 4.2) Data Availability & Coverage Analysis

In [ ]:
all_metrics = [metric for metric in monet_data.columns]
irrelevant_metrics = [metric for metric in pipeline.stages[3].additional_results["irrelevant_metrics"].columns]
sparse_metrics = [col_idx.zfill(14) for col_idx, col in monet_data.items() if col.count()<10]
kept_metrics = [metric for metric in monet_clean.columns]
pruned_metrics = set(all_metrics) - set(kept_metrics)

irrelevant_metrics_df = metric2capital_map.loc[metric2capital_map.index.isin(irrelevant_metrics)]
sparse_metrics_df = metric2capital_map.loc[metric2capital_map.index.isin(sparse_metrics)]
kept_metrics_df = metric2capital_map.loc[metric2capital_map.index.isin(kept_metrics)]
pruned_metrics_df = metric2capital_map.loc[metric2capital_map.index.isin(pruned_metrics)]

#### 4.2.1) Analysis of capitals

Question: How many metrics per capital are there?

In [ ]:
metrics_meta_table = metric2capital_map[metric2capital_map.index.str.endswith("metr")]

In [ ]:
n_metrics_per_cap_all = metrics_meta_table.groupby("capital - primary").agg("count")
n_metrics_per_cap_cleaned = metrics_meta_table.loc[metrics_meta_table.index.isin(kept_metrics),:].groupby("capital - primary").agg("count")

In [ ]:
n_metrics_per_cap = n_metrics_per_cap_all\
                    .rename({"metric_name": "before cleaning"}, axis=1)\
                    .join(n_metrics_per_cap_cleaned.rename({"metric_name": "after cleaning"}, axis=1))

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
n_metrics_per_cap.plot(kind="bar", ax=ax)
ax.grid(True)
ax.set_title("Number of Metrics per Capital")
ax.set_xticks([0,1,2,3], n_metrics_per_cap.index, rotation=0)
ax.set_ylabel("Count")
plt.show()

#### 4.2.2) Analysis of sparse data

Question: How many metrics per capital are too sparse (i.e. have less than 10 data points)

In [ ]:
sparse_metrics_df = sparse_metrics_df.join(monet_data.count(axis=0)
                                           .to_frame()
                                           .rename({0: "count"}, axis=1),
                                           how="left"
                                          )
sparse_metrics_df = sparse_metrics_df.sort_values(by="count", ascending=False)
sparse_metrics_df.to_csv(const.sparse_metrics_analysis_fpath) 
sparse_metrics_df.head()

In [ ]:
sparse_metrics_per_capital = sparse_metrics_df.groupby("capital - primary").agg({"count": "count"})

fig, ax = plt.subplots(figsize=(8,3))
sparse_metrics_per_capital.plot(kind="bar", ax=ax, legend=False)
ax.grid(True)
ax.set_xlabel("capital")
ax.set_ylabel("count")
ax.set_title("Number of removed metrics per capital due to insufficient data availability")
plt.xticks(rotation=0)
plt.tight_layout()
fig.savefig(const.n_sparse_by_capital_plot_fpath)
plt.show()

#### 4.2.3) Analysis of irrelevant data

Question: How many metrics per capital are irrelevant to agenda2030?

In [ ]:
irrelevant_metrics_df = irrelevant_metrics_df.join(monet_data.count(axis=0)
                                                   .to_frame()
                                                   .rename({0: "count"}, axis=1),
                                                   how="left"
                                                  )
irrelevant_metrics_df = irrelevant_metrics_df.sort_values(by="count", ascending=False)
irrelevant_metrics_df.to_csv(const.irrelevant_metrics_analysis_fpath)
irrelevant_metrics_df.head()

In [ ]:
irrelevant_metrics_per_capital = irrelevant_metrics_df.groupby("capital - primary").agg({"count": "count"})

fig, ax = plt.subplots(figsize=(8,3))
irrelevant_metrics_per_capital.plot(kind="bar", ax=ax, legend=False)
ax.grid(True)
ax.set_xlabel("capital")
ax.set_ylabel("count")
ax.set_title("Number of metrics irrelevant to agenda2030 (per capital)")
plt.xticks(rotation=0)
plt.tight_layout()
fig.savefig(const.n_irrelevant_by_capital_plot_fpath)
plt.show()

### 4.3) Analysis of raw data availability

Goal: Visually represent how many data points are available for each metric. Add additional information such as when that data is available and what capital the metric belongs to. 

In [ ]:
datapoint_counts = monet_data.count()\
                             .sort_values(ascending=False)\
                             .to_frame()\
                             .rename({0: "count"}, axis=1)
metric_availability = datapoint_counts.join(metric2capital_map)
metric_availability = metric_availability.dropna(subset=["capital - primary"])

In [ ]:
fig, ax = plt.subplots(figsize=(17,30))
plot.raw_data_availability_barchart(metric_availability,
                                    "counts",
                                    "Metric Name",
                                    "Number of measured data points per metric",
                                    ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
monet_trp = monet_data.transpose()
monet_trp = monet_trp.loc[:,~monet_trp.columns.duplicated()].copy()

# Get existing years from column index (assumed datetime index)
existing_years = [col for col in monet_trp.columns]
full_year_range = list(range(min(existing_years), max(existing_years) + 1))

# Identify missing years
missing_years = [y for y in full_year_range if y not in existing_years]

# Add missing columns with white color
for year in missing_years:
    monet_trp[year] = np.nan * len(monet_trp)

# Reorder columns chronologically
monet_trp = monet_trp.reindex(sorted(monet_trp.columns), axis=1)

monet_trp["capital"] = metric2capital_map.loc[monet_trp.index, "capital - primary"]

In [ ]:
df_plot = monet_trp.loc[metric_availability.index,:]

fig,axs=plt.subplots(1,2, figsize=(17,30), sharey=True, gridspec_kw = {"wspace": 0.01})
# Left plot panel
plot.visualize_data_availability_colored(df_plot,
                                         "Year",
                                         "Metric Name",
                                         "MONET2030 metric data availability across time",
                                         ax=axs[0]
                                        )

# Right plot panel
plot.raw_data_availability_barchart(metric_availability,
                                    "Number of measured data points per metric",
                                    "Metric Name",
                                    "Number of measured data points per metric",
                                    ax=axs[1],
                                    show_legend=False
                                    )
plt.tight_layout()
fig.suptitle("Data availability", y=0.9, fontsize=18)
fig.savefig(const.data_availability_dir / "data_availability_all.pdf", bbox_inches="tight")
plt.show()

### 4.4) Correlation Analysis

Question: Which metrics are redundant?

In [ ]:
monet_ca = utils.CorrelationAnalysis(final_output)
zero_lag_corrmat = monet_ca.cross_corr(lag=0)
agg_corrmat = monet_ca.max_abs_corr()

In [ ]:
for th in [0.8, 0.85, 0.9, 0.95, 0.99]:
    monochronic_fpath = const.corra_dir/f"keep_monochron_corr_th{str(int(th*100))}.xlsx"
    diachronic_fpath = const.corra_dir/f"keep_diachron_corr_th{str(int(th*100))}.xlsx"
    
    to_keep_zero_lag, corr_xlsx_zero_lag = monet_ca.drop_strong_correlations(zero_lag_corrmat, 
                                                                             threshold=th, 
                                                                             id2name_map = metric2capital_map,
                                                                             fpath_corr=monochronic_fpath)
    
    to_keep_agg, corr_xlsx_agg = monet_ca.drop_strong_correlations(agg_corrmat,
                                                                   threshold=th, 
                                                                   id2name_map = metric2capital_map,
                                                                   fpath_corr = diachronic_fpath)

In [ ]:
th_vec = [th/100 for th in range(80,100,2)]+[0.99, 0.999]

counts = pd.DataFrame(index=["Social", "Human", "Natural", "Economic"], columns=[])

for th in th_vec:
    to_keep, _ = monet_ca.drop_strong_correlations(agg_corrmat, threshold=th)
    counts_per_cap = metric2capital_map[metric2capital_map.index.isin(to_keep)].groupby("capital - primary").agg("count")
    counts = counts.join(counts_per_cap.rename({"metric_name": th}, axis=1), how="outer")
    
counts.loc["Total", :] = counts.sum()
counts

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
counts.T.plot(kind="line", ax=ax)
ax.set_xticks(th_vec, th_vec, rotation=60)
ax.set_yticks(range(1,42,2), range(1,42,2))
ax.set_xlabel("correlation threshold")
ax.set_ylabel("Number of non-redundant metrics")
ax.set_title("Counting non-redundant metrics in dependency of\ncorrelation threshold")
ax.grid()
plt.tight_layout()
fig.savefig(const.corra_dir / "n_nonredundant_per_threshold.pdf", bbox_inches="tight")

In [ ]:
metric2capital_map[metric2capital_map.index.isin(to_keep_agg)].groupby("capital - primary").agg("count")

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,4))
ax = axs[0]
sns.histplot(zero_lag_corrmat.unstack().values, ax=ax, kde=True, bins=40)
ax.set_title("Distribution of correlation values")
ax.set_xlabel("corr")

ax = axs[1]
sns.histplot(zero_lag_corrmat.unstack().abs().values, ax=ax, bins=20)
ax.set_title("Distribution of abs(correlation) values")
ax.set_xlabel("abs(corr)")
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,4))
ax = axs[0]
sns.histplot(agg_corrmat.unstack().values, ax=ax, kde=True, bins=40)
ax.set_title("Distribution of correlation values")
ax.set_xlabel("corr")

ax = axs[1]
sns.histplot(agg_corrmat.unstack().abs().values, ax=ax, bins=20)
ax.set_title("Distribution of abs(correlation) values")
ax.set_xlabel("abs(corr)")
plt.tight_layout()
plt.show()

### 4.5) Trend & Performance Analysis

#### 4.5.1) Analysis of trend statistics per capital
Question: How well do the individual capitals do in terms of evolving in the right direction?

In [ ]:
# Get a table with all metrics including their name and desired_trend and 
# prepare two columns for slope information: slope and slope_norm
directions = pd.read_csv(const.trend_directions).set_index("metric_id")
directions["dam_id"] = [int(metr_id.split("_")[0][:-1]) for metr_id in directions.index]
directions_with_names = directions.join(metric2capital_map[["metric_name"]])

performer_ranking = directions_with_names.copy()
performer_ranking[["slope", "slope_norm"]] = None
performer_ranking.head()

In [ ]:
# Fill in the slope and slope_norm columns

# REMARK: slope_norm is introduced the put all metrics
# on a common scale. Depending on the desired_trend, for
# some metrics a higher (i.e. more positive) slope is 
# better, for other metrics a lower (i.e. more negative)
# slope is better. The column slope_norm simplifies this
# as a higher (more positive) slope_norm is always better.

for mid in performer_ranking.index:
    data = monet_data[mid].dropna()
    x = [year for year in data.keys()]
    y_norm = data.values/data.values[0]
    slope = np.polyfit(x,y_norm,1)[0]
    performer_ranking.loc[mid, "slope"] = slope
        
    if performer_ranking.loc[mid, "desired_trend"]=="up":
        performer_ranking.loc[mid, "slope_norm"] = performer_ranking.loc[mid, "slope"]
    elif performer_ranking.loc[mid, "desired_trend"]=="down":
        performer_ranking.loc[mid, "slope_norm"] = -performer_ranking.loc[mid, "slope"]
    else:
        performer_ranking.loc[mid, "slope_norm"] = np.nan
    
performer_ranking = performer_ranking.join(metric2capital_map[["capital - primary"]])
performer_ranking = performer_ranking.dropna(subset="slope_norm").sort_values(by="slope_norm", ascending=False)
performer_ranking

In [ ]:
# Get summary statistics about the slope_norm
slope_norm_stats = performer_ranking.groupby("capital - primary").agg({"slope_norm": ["mean", "median", "std"]})
slope_norm_stats.to_csv(const.slope_stats_fpath)
slope_norm_stats

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
sns.boxplot(data=performer_ranking, x="capital - primary", y="slope_norm", ax=ax)
ax.scatter([2],[-0.045], marker="v", color="red", label="direction of single extreme outlier")
ax.legend(loc="upper right")
ax.grid(True)
ax.set_ylim([-0.05,0.3])
ax.set_title("Distribution of normalized slopes per capital")
fig.savefig(const.slope_distro_plot_fpath)
plt.show()

#### 4.5.2) Best & Worst Performering Metrics
##### 4.5.2.a) Best & Worst Performering Metrics over all
Question: Which are the 3 best and worst performing metrics over all capitals?

In [ ]:
ranked_metatable = performer_ranking.reset_index()\
                                    .merge(scraper_pipeline.metatable.drop(["observable", 
                                                                            "description", 
                                                                            "capital - primary", 
                                                                            "data_file_url", 
                                                                            "hyperlink", 
                                                                            "units", 
                                                                            "id"
                                                                           ], axis=1),
                                           on="dam_id"
                                          )
ranked_metatable = ranked_metatable[["metric_id", 
                                     "metric_name", 
                                     "capital - primary", 
                                     "sdg", 
                                     "topic", 
                                     "indicator_id",
                                     "observable", 
                                     "description",
                                     "is_key",
                                     "slope_norm",
                                     "desired_trend",
                                     "slope"
                                    ]].set_index("metric_id")
ranked_metatable

In [ ]:
df = ranked_metatable
fig, ax = plt.subplots(figsize=(15,30))
# Reset index for plotting
df_plot = df.reset_index()
# Use a numeric x axis to avoid categorical spacing issues
df_plot["y_pos"] = range(len(df_plot))
    
# Plot bars
sns.barplot(
    data=df_plot,
    x="slope_norm", y="y_pos", hue="capital - primary",
    dodge=False,  # Keep all hues at the same x-position
    orient="horizontal",
    ax=ax,
    legend=True
)

# Optional: show fewer x-ticks
yticks = df_plot["y_pos"]
ylabels = df_plot["metric_name"]
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels, fontsize=8)

# Labels and title
ax.set_xlabel("normalized slope (higher = better)")
ax.set_ylabel("metric names")
ax.set_title("Ranking of evolution over time of MONET2030 metrics")
ax.grid(True)

plt.tight_layout()
fig.savefig(const.performance_ranking_plot_fpath)
plt.show()

In [ ]:
columns = ["metric_name", 
           "capital - primary", 
           "sdg", 
           "topic", 
           "indicator_id", 
           "observable", 
           "description", 
           "is_key", 
           "slope", 
           "slope_norm", 
           "desired_trend"]

In [ ]:
top3 = ranked_metatable.sort_values(by="slope_norm", ascending=False).head(3)
top3 = top3[columns]
top3.to_excel(const.top3_metrics_fpath)
top3

In [ ]:
bottom3 = ranked_metatable.sort_values(by="slope_norm", ascending=False).tail(3)
bottom3 = bottom3[columns]
bottom3.to_excel(const.bottom3_metrics_fpath)
bottom3

##### 4.5.2.a) Best & Worst Performering Metrics per capital
Question: Which are the 3 best and worst performing metrics within each capital?

In [ ]:
print("TOP 3 PERFORMERS PER CAPITAL")
print("============================")
n_top_key = 0
n_top_tot = 0
with pd.ExcelWriter(const.top3_metrics_per_cap_fpath) as writer:
    for cap in four_caps:
        print(cap)
        print(len(cap)*"-")
        perf_cap = ranked_metatable[ranked_metatable["capital - primary"]==cap][columns]
        top3 = perf_cap.head(3)
        n_top_key += top3["is_key"].sum()
        n_top_tot += top3["is_key"].count()
        display(top3)
        top3.to_excel(writer, sheet_name=cap)

In [ ]:
print("BOTTOM 3 PERFORMERS PER CAPITAL")
print("============================")
#columns = ["metric_name", "indicator_id", "sdg", "observable", "description", "capital - primary", "is_key", "slope", "slope_norm", "desired_trend"]

n_bottom_key = 0
n_bottom_tot = 0
with pd.ExcelWriter(const.bottom3_metrics_per_cap_fpath) as writer:
    for cap in four_caps:
        print(cap)
        print(len(cap)*"-")
        perf_cap = ranked_metatable[ranked_metatable["capital - primary"]==cap][columns]
        bottom3 = perf_cap.tail(3)
        display(bottom3)
        n_bottom_key += bottom3["is_key"].sum()
        n_bottom_tot += bottom3["is_key"].count()
        bottom3.to_excel(writer, sheet_name=cap)

#### 4.5.3) Best & Worst Performing Key Indicators 
Question: How many key indicators are there among the groups of best and worst performing metrics, respectively?

In [ ]:
xs = np.array([1,2])
n_key = np.array([n_top_key, n_bottom_key])
n_tot = np.array([n_top_tot, n_bottom_tot])

fig, ax = plt.subplots()
ax.bar([1,2], n_key, facecolor="blue", label="key indicators")
ax.set_xticks([1,2], ["best-performing", "worst-performing"])
ax.grid(True)
ax.set_title("Number of key indicators within best- & worst-performing indicators\n(summed over all capitals)")
fig.savefig(const.n_key_indicators_per_performance_plot_fpath)
plt.show()

Question: What is the ranking of only the key indicators with respect to their evolution over time?

In [ ]:
key_ind_trend = scraper_pipeline.key_indicators_df.merge(ranked_metatable, left_on="id", right_on="indicator_id", how="left")[["id", "indicator", "capital - primary", "slope_norm"]]
key_ind_trend = key_ind_trend.sort_values(by="slope_norm", ascending=False).dropna(subset="slope_norm")
key_ind_trend["rank"] = range(1, len(key_ind_trend)+1)
key_ind_trend.to_excel(const.key_indicator_performance_fpath)
key_ind_trend